In [1]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.isri import ISRIStemmer
import spacy
import string
# from google.cloud import vision, translate
# from google.cloud.vision import types
# import google.auth
# from protobuf_to_dict import protobuf_to_dict
# from google.protobuf.json_format import MessageToJson
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import gensim
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full

from collections import OrderedDict
from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import row, column
import random


import importlib
ldavis = importlib.util.find_spec("pyLDAvis")
import pyLDAvis.gensim as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()
# print(pyLDAvis.enable_notebook())

In [3]:
# nlp = spacy.load('en_core_web_lg')
nlp = spacy.load('xx')

In [4]:
ar_df = pd.read_csv('arabic2_test.csv', names=['file_name', 'page_number', 'google_vision_text', 'google_translate_text'])

In [4]:
ar_df

,file_name,page_number,google_vision_text,google_translate_text
0,image_lucida_app/media/Data_RepositoryEgyptian...,0,العدد ۱۷۷۲ - السنة السابعة والثلاثون\nالاثنين...,Number 1772 - Thirty-seventh year Monday 28 Ma...
1,image_lucida_app/media/Data_RepositoryEgyptian...,4,دولارا لیں صودا\nالنكون عون القوالبراعم الواه...,"3 - clarity in the vision of goals, and follow..."
2,image_lucida_app/media/Data_RepositoryEgyptian...,5,اللا کے\nت روما\nعضو اللجنة السياسية\nعزیز صد...,Al-Lajka Rome Member of the Political Committe...
3,image_lucida_app/media/Data_RepositoryEgyptian...,6,تمرير الخبراء الأبد\nمشروعات والعب في البر لز...,Experts forever pass projects and play on land...
4,image_lucida_app/media/Data_RepositoryEgyptian...,9,5\n*\nميثاق الملك حسين !! .\nويشكل خلال الأسا...,5 * The Charter of King Hussein !! . Within th...
5,image_lucida_app/media/Data_RepositoryEgyptian...,1,ملاحظات حول الميثان)\nاقوا النار على مستوى ال...,Notes about methane) Fire the level of Constit...
6,image_lucida_app/media/Data_RepositoryEgyptian...,10,رویه\nا\nكيف تقرأ الميثاق.\nی\nعم السعرالسای\...,See how to read the Charter. The price of the ...
7,image_lucida_app/media/Data_RepositoryEgyptian...,11,-\nفي هااطی ۰.را نفت قانون اراصاع الزای\nهل ه...,- In Hatti 0.37 The law of Ara&#39;aa al-Zayi ...
8,image_lucida_app/media/Data_RepositoryEgyptian...,3,بهیارفل دلکویرالطوارجاعه\nالى حد انیبال لبنان...,"No, certainly not .. And it happened that the ..."
9,image_lucida_app/media/Data_RepositoryEgyptian...,12,أحكااال\nایزنی\nفي تمام الثامنة من مساء الاثن...,"Ahkala Eizni At 8:00 pm last Monday, a major e..."


In [6]:
st = ISRIStemmer()
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()
#     print('text', st.stem(text))
    return text

def custom_tokenize(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text(df):
    doc = []
    final_doc = []
    for index, row in df.iterrows():
        raw_text = row['google_vision_text']
        tokens = clean_str(raw_text)
        text = custom_tokenize(tokens)
        page_terms = ''
        for t in text: 
            if t in stopwords.words('arabic'):
                continue
            else:
#                 print(t)
                page_terms += t + ' '
            doc.append(page_terms)
#     print(doc)
    return doc


    

In [9]:
texts = process_text(ar_df)
# f = open('arabic_test.txt', 'wt', encoding='utf-8')
# f.write(texts)
print(texts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [8]:
set(nltk.corpus.stopwords.words("arabic"))

{'آه',
 'آها',
 'آي',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'أم',
 'أما',
 'أن',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنى',
 'أو',
 'أولئك',
 'أولاء',
 'أوه',
 'أي',
 'أين',
 'أينما',
 'أيها',
 'إذ',
 'إذا',
 'إذما',
 'إذن',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إما',
 'إن',
 'إنا',
 'إنما',
 'إنه',
 'إي',
 'إيه',
 'التي',
 'الذي',
 'الذين',
 'اللائي',
 'اللاتي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بيد',
 'بين',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل',
 'على',
 'عليك',
 'عليه',
 'عم

In [7]:
def custom_tokenize_english(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text_english(df, types):
    doc = []
    final_doc = []
    for index, row in df.iterrows():
        raw_text = row['google_translate_text']
        tokens = custom_tokenize_english(raw_text)
        page_terms = ''
        for t in tokens:
            if t in string.punctuation:
                pass
            elif t in stopwords.words('english'):
                pass
            else:
                page_terms += t.lower() + ' '
        doc.append(page_terms)
    print(doc)
    for sent in doc:
        sent_terms = ''
        spacy_text = nlp(sent)
        for ent in spacy_text.ents:
            if ent.label_ in types:
                sent_terms += ent.text + ' '
        final_doc.append(sent_terms)
    return final_doc

In [8]:
order_types = ['GPE', 'NORP', 'PERSON']
test = process_text_english(ar_df, order_types)
test

['number 1772 thirty-seventh year monday 28 may 1992 price 40 mm 52 e m 2 quot they except national charter engineer gamal abdel nase o bashmehndg gap us detailed drawing villa ', '3 clarity vision goals follow-up constantly avoid emotional drift sub-paths move away national struggle threaten part the three items define basic national claim also principles revolutionary philosophy revolutionary theory they define principles general strategic plan define principles detailed plan ie tactic he fall c principles extend partial details applications charter reached definition principles beyond 00 reached distant parts for example charter production management happens said quot it necessary us know written word basket among easy memorize future it also completes important link connection idea experiment those responsible national action write ideas front responsible implementation etc this principle although principle concerning details labor administration another principle organization must

['',
 '',
 'british kamal al-din hassin cairo nkrumah british british british beirut turks ataturk » cairo washington al yousif ',
 'arab american arab arab american al-yousef american arab arab america germany japan germany germany canada india arab arab arab al-yusuf al-yusuf ',
 'cairo american missouri american cairo washington arab cuba cuban american cairo british germany cairo british kuwait american cairo paris cairo cairo al-youssef ',
 '',
 'al nader • abdul-asmar al yousef ',
 '',
 'karl karl marx al-yusuf ',
 'ahmad daoud arabs mahmoud al-nallah al-hafiz falah al-atta al yousuf al yousuf mohamed ali ahmed al yousef mohamed al-yusuf ',
 'italian cairo alexandria czech cairo al-aini al-ayni ',
 'al-khayan arab ras sal ',
 'al-youssef joseph al el youssef 1771 ',
 'mohamed ali ahmed al-hai al-atta mahmud hussein » bahri al-sayed al-yusuf al-atta ',
 'albanians al-hayy al-atta al-hayy al-atta italians al yusuf ',
 '',
 'kurds ',
 'mustafa quot » st mustafa shand mustafa fatima 